In [ ]:
!pip install "matplotlib>=3.2,<=3.7.3"
!pip install "ydata-profiling>4.4,<4.5"
!pip install tensorflow sdv

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

2023-11-02 14:45:29.847332: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 14:45:29.908582: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 14:45:29.908623: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 14:45:29.908662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 14:45:29.919297: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 14:45:29.920217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

## Ideas

If adversary does not work, predict -> predict_proba to get more information

In [2]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from pia_functions import data_train_test, get_distributed_adult_sets, generate_shadow_model_outputs

### Generate training data for shadow models

In [ ]:
n_shadow_models=200
distributions=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
X_train, X_test, y_train, y_test, sensitive, sensitive_t = data_train_test()

In [13]:
model_input

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,69,70,71,72,73,74,75,76,77,78
0,0,0.312424,0.000000,0.0,0.094047,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1,0.397083,0.004028,0.0,0.399298,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2,0.801682,0.000000,0.0,0.499176,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,0.421440,0.003712,0.0,0.394823,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,0.419976,0.000000,0.0,0.392910,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.817372,0.000000,0.0,0.398106,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9996,9996,0.534707,0.000000,0.0,0.393522,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9997,9997,0.794512,0.160842,0.0,0.402341,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9998,9998,0.213765,0.003364,0.0,0.397423,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [15]:
model_input = pd.read_csv("data/syn_data.csv", names=X_train.columns)

In [ ]:
distributed_datasets = get_distributed_adult_sets(distributions=distributions)

## Generate synthetic data for model output

In [4]:
output_size = 10000
metadata = SingleTableMetadata()
to_fit = pd.DataFrame(np.concatenate((X_train, X_test)), columns=[str(i) for i in range(79)])
#to_fit = to_fit.astype({i: 'int' for i in range(4,79)})
#to_fit["index"] = to_fit.index
#to_fit = to_fit.astype({"index": 'string'})
metadata.detect_from_dataframe(data=to_fit)

In [7]:
metadata.to_dict()

{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'0': {'sdtype': 'numerical'},
  '1': {'sdtype': 'numerical'},
  '2': {'sdtype': 'numerical'},
  '3': {'sdtype': 'numerical'},
  '4': {'sdtype': 'categorical'},
  '5': {'sdtype': 'categorical'},
  '6': {'sdtype': 'categorical'},
  '7': {'sdtype': 'categorical'},
  '8': {'sdtype': 'categorical'},
  '9': {'sdtype': 'categorical'},
  '10': {'sdtype': 'categorical'},
  '11': {'sdtype': 'categorical'},
  '12': {'sdtype': 'categorical'},
  '13': {'sdtype': 'categorical'},
  '14': {'sdtype': 'categorical'},
  '15': {'sdtype': 'categorical'},
  '16': {'sdtype': 'categorical'},
  '17': {'sdtype': 'categorical'},
  '18': {'sdtype': 'categorical'},
  '19': {'sdtype': 'categorical'},
  '20': {'sdtype': 'categorical'},
  '21': {'sdtype': 'categorical'},
  '22': {'sdtype': 'categorical'},
  '23': {'sdtype': 'categorical'},
  '24': {'sdtype': 'categorical'},
  '25': {'sdtype': 'categorical'},
  '26': {'sdtype': 'categorical'},
  '27': {'sdtype'

In [5]:
syn_model = CTGANSynthesizer(metadata)
syn_model.fit(to_fit)

/home/js/anaconda3/envs/adversarial-bb-training/lib/python3.10/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column '0'. Data will not be rounded.
  warnings.warn(
/home/js/anaconda3/envs/adversarial-bb-training/lib/python3.10/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column '1'. Data will not be rounded.
  warnings.warn(
/home/js/anaconda3/envs/adversarial-bb-training/lib/python3.10/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column '2'. Data will not be rounded.
  warnings.warn(
/home/js/anaconda3/envs/adversarial-bb-training/lib/python3.10/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column '3'. Data will not be rounded.
  warnings.warn(


In [7]:
sampled = syn_model.sample(num_rows=output_size)
syn_model.save('syn_model')

In [14]:
sampled.to_csv("data/syn_data.csv", index=False)

### Train shadow models and generate output

In [ ]:
all_shadow_outputs = []
for ds in distributed_datasets:
    outputs = generate_shadow_model_outputs(ds, X_test, n_shadow_models=n_shadow_models)
    all_shadow_outputs.append(outputs)

### Save shadow model outputs to file

In [ ]:
adv_df = pd.DataFrame(np.array(np.concatenate((all_shadow_outputs))))
adv_df["y"] = np.concatenate(([np.repeat(d, n_shadow_models) for d in distributions]))
adv_df.to_csv("data/shadow_model_outputs.csv")

### Shadow model outputs as training data for adversary

In [ ]:
adv_df = pd.read_csv("data/shadow_model_outputs.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [75]:
adv_X.shape

(1800, 9770)

### Create adversary

In [73]:
adversary = keras.Sequential()
adversary.add(keras.Input(shape=(adv_X.shape[1],)))
adversary.add(keras.layers.Dense(8, activation='relu'))
adversary.add(keras.layers.Dense(4, activation='relu'))
adversary.add(keras.layers.Dense(1))

adversary.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

### Train adversary

In [74]:
adversary.fit(adv_X, adv_y, validation_split=0.2, batch_size=32, epochs=10)

Epoch 1/10
45/45 [==============================] - 1s 8ms/step - loss: 0.7688 - val_loss: 0.0686
Epoch 2/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0099 - val_loss: 0.0017
Epoch 3/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0021 - val_loss: 0.0014
Epoch 4/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0018 - val_loss: 0.0013
Epoch 5/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 6/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 7/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 8/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0014 - val_loss: 0.0010
Epoch 9/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0013 - val_loss: 0.0010
Epoch 10/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0012 - val_loss: 9.4432e-04


# Test adversary

In [76]:
adv_df_test = pd.read_csv("data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True)
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [80]:
adversary.evaluate(adv_X_test, adv_y_test)#, adv_y_test)

15/15 [==============================] - 0s 2ms/step - loss: 0.1105


0.11049496382474899

In [79]:
adv_y_test

0      0.9
1      0.8
2      0.4
3      0.2
4      0.1
      ... 
445    0.6
446    0.2
447    0.8
448    0.5
449    0.1
Name: y, Length: 450, dtype: float64

### Next steps
* Use adversary during training

### (Data set analysis)

In [ ]:
import ydata_profiling

ydata_profiling.ProfileReport(X_test)

In [ ]:
# RANDOM DS
random_dict = {}
for attr in ds.X_train:
    print(f'{round(X_train_pr[attr].min(),2)} {round(X_train_pr[attr].max(),2)}')
    random_attr = np.random.rand(random_length)*(X_train_pr[attr].max()+abs(X_train_pr[attr].min()))-abs(X_train_pr[attr].min())
    random_dict[attr] = random_attr

In [ ]:
random_ds = pd.DataFrame(random_dict)